In [31]:
import tensorflow as tf
import numpy as np
import os
import random
import pandas as pd
import seaborn as sns
from datetime import datetime
import matplotlib.pyplot as plt
plt.rc('font', size=16)
from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')
tf.get_logger().setLevel('ERROR')

tfk = tf.keras
tfkl = tf.keras.layers
print(tf.__version__)

In [32]:
# Random seed for reproducibility
seed = 42

random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.compat.v1.set_random_seed(seed)

In [33]:
df = pd.read_csv("../input/training/Training.csv")
print(df.shape)
df.head()

In [34]:
sns.set(rc={"figure.figsize":(12, 6)})
sns.lineplot(data=df[:1000])

From the plot we can see that there are some features that show similar trends. We can perform a deeper analysis by looking at their correlation. 

In [35]:
def inspect_dataframe(df, columns):
    figs, axs = plt.subplots(len(columns), 1, sharex=True, figsize=(17,17))
    for i, col in enumerate(columns):
        axs[i].plot(df[col])
        axs[i].set_title(col)
    plt.show()
inspect_dataframe(df, df.columns)

In [36]:
df.describe().transpose()

In [37]:
df_std = df
df_std = df_std.melt(var_name='Column', value_name='Not Normalized')
plt.figure(figsize=(12, 6))
ax = sns.violinplot(x='Column', y='Not Normalized', data=df_std)
_ = ax.set_xticklabels(df.columns, rotation=90)

We see that the dataset is not normalized. 

# Correlation Analysis

In [38]:
from scipy.stats import pearsonr

def corrfunc(x,y, ax=None, **kws):
    """Plot the correlation coefficient in the top left hand corner of a plot."""
    r, _ = pearsonr(x, y)
    ax = ax or plt.gca()
    # Unicode for lowercase rho (ρ)
    rho = '\u03C1'
    ax.annotate(f'{rho} = {r:.2f}', xy=(.1, .9), xycoords=ax.transAxes)
    
def corrdot(*args, **kwargs):
    corr_r = args[0].corr(args[1], 'pearson')
    corr_text = f"{corr_r:2.2f}".replace("0.", ".")
    ax = plt.gca()
    ax.set_axis_off()
    marker_size = abs(corr_r) * 10000
    ax.scatter([.5], [.5], marker_size, [corr_r], alpha=0.6, cmap="Blues",
               vmin=-1, vmax=1, transform=ax.transAxes)
    font_size = abs(corr_r) * 40 + 5
    ax.annotate(corr_text, [.5, .5,],  xycoords="axes fraction",
                ha='center', va='center', fontsize=font_size)    
    
# g = sns.pairplot(stocks,palette=["Blues_d"])
g = sns.PairGrid(df, aspect=1.4, diag_sharey=False)
g.map_lower(corrfunc)
g.map_lower(sns.regplot, lowess=True, ci=False, line_kws={'color': 'Black','linewidth':1})
g.map_diag(sns.distplot, kde_kws={'color': 'Black','linewidth':1})
g.map_upper(corrdot)
plt.show()

'Crunchiness' and 'Hype root' have an high correlation coefficient. This means that they are strongly correlated, in the sense that if one of them will increase, also the other will be prone to increase. Same reasonment holds for 'Wonder Level' and 'Loudiness on impact'.

In [39]:
#drop crunchiness (hype root remains) and Wonder level (loudiness on impact remains)
df= df.drop(['Crunchiness', 'Wonder level'], axis=1)
print(df)

In [40]:
#normalization
scaler = MinMaxScaler()
ndf = pd.DataFrame(scaler.fit_transform(df),columns=df.columns)

In [41]:
ndf.describe().transpose()

In [42]:
df_std = (ndf - ndf.mean()) / ndf.std()
df_std = df_std.melt(var_name='Column', value_name='Normalized')
plt.figure(figsize=(12, 6))
ax = sns.violinplot(x='Column', y='Normalized', data=df_std)
_ = ax.set_xticklabels(ndf.columns, rotation=90)

In [43]:
test_size = 3500 #dei 68528 campioni che ho, considero gli utlimi 3500 come test
X_train_raw = ndf.iloc[:-test_size]
X_test_raw = ndf.iloc[-test_size:]
print(X_train_raw.shape, X_test_raw.shape)


for col,cont in ndf.iteritems():
    plt.figure(figsize=(17,5))
    plt.plot(cont, label='Train {}'.format(col))
    plt.plot(X_test_raw[col], label='Test {}'.format(col))
    plt.title('Train-Test Split')
    plt.legend()
    plt.show()

In [44]:
print(X_train_raw.shape,X_test_raw.shape)

In [45]:
def build_sequences(df, target_labels, window, stride, telescope):
  #the telescope tells us how many parameters i want to predict in the future
    # Sanity check to avoid runtime errors
    assert window % stride == 0
    dataset = []
    labels = []
    temp_df = df.copy().values
    #target_labels allows to choose how many sensors we are going to predict in the future
    temp_label = df[target_labels].copy().values
    padding_len = len(df)%window

    if(padding_len != 0):
        # Compute padding length
        padding_len = window - len(df)%window
        padding = np.zeros((padding_len,temp_df.shape[1]), dtype='float64')
        temp_df = np.concatenate((padding,df))
        padding = np.zeros((padding_len,temp_label.shape[1]), dtype='float64')
        temp_label = np.concatenate((padding,temp_label))
        assert len(temp_df) % window == 0

    for idx in np.arange(0,len(temp_df)-window-telescope,stride):
        dataset.append(temp_df[idx:idx+window])
        labels.append(temp_label[idx+window:idx+window+telescope])

    dataset = np.array(dataset)
    labels = np.array(labels)
    return dataset, labels

# Training: one step prediction

In [46]:
# training dataset parameters
window=300 # size of the set of samples we're using to train the NN
stride=10 # space between the beginning of one window and the next
telescope=864 # How many steps to predict in the future
target_labels = ndf.columns

In [47]:
X_min = X_train_raw.min()
X_max = X_train_raw.max()
future = df[-window:]
future = (future-X_min)/(X_max-X_min)
future = np.expand_dims(future, axis=0)
future.shape

In [48]:
X_train, y_train = build_sequences(X_train_raw, target_labels, window, stride, telescope)
X_test, y_test = build_sequences(X_test_raw, target_labels, window, stride, telescope)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

In [49]:
def inspect_multivariate(X, y, columns, telescope, idx=None):
    if(idx==None):
        idx=np.random.randint(0,len(X))

    figs, axs = plt.subplots(len(columns), 1, sharex=True, figsize=(17,17))
    for i, col in enumerate(columns):
        axs[i].plot(np.arange(len(X[0,:,i])), X[idx,:,i])
        axs[i].scatter(np.arange(len(X[0,:,i]), len(X_train[0,:,i])+telescope), y[idx,:,i], color='orange')
        axs[i].set_title(col)
        axs[i].set_ylim(0,1)
        axs[i].legend()
    plt.show()

In [50]:
inspect_multivariate(X_train, y_train, target_labels, telescope)

In [51]:
# model training parameters
input_shape = X_train.shape[1:]
output_shape = y_train.shape[1:]
batch_size = 32
epochs = 200

# Models

In [52]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, LSTM, Dense, Reshape, Conv1D, MaxPool1D

In [53]:
models = {}
results = {}

In [54]:
models['modeldropped'] = Sequential([
    GRU(256, input_shape = input_shape, return_sequences=False,name="gru1"),
    
    Dense(output_shape[-1]*output_shape[-2], activation='relu', name="dense"),
    Reshape((output_shape[-2],output_shape[-1]), name="reshape_to_batch"),
    Conv1D(output_shape[-1], 1, padding='same', name="finalconv"),
])


In [55]:
for model in models.values():
    model.compile(loss=tfk.losses.MeanSquaredError(), optimizer=tfk.optimizers.RMSprop(), metrics=['mae'])
    model.summary()
    print('***\n\n')

In [ ]:
for key, model in models.items():
    results[key] = model.fit(
        x = X_train,
        y = y_train,
        batch_size = batch_size,
        epochs = epochs,
        validation_split=.2,
        callbacks = [
           tfk.callbacks.EarlyStopping(monitor='val_loss', mode='min', patience=20, restore_best_weights=True),
           tfk.callbacks.ReduceLROnPlateau(monitor='val_loss', mode='min', patience=5, factor=0.5, min_lr=1e-4)
        ]
    ).history

In [ ]:
for key, model in models.items():
    model.save(f'models/{key}')

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(16, 24))

colors = ['red', 'green', 'blue', 'orange']

for index, (key, history) in enumerate(results.items()):
    best_epoch = np.argmin(history['val_loss'])

    ax[0].plot(history['loss'], label=f'{key} Train. loss', alpha=.9, color=colors[index])
    ax[0].plot(history['val_loss'], label=f'{key} Valid. loss', alpha=.9, color=colors[index], ls='--')
    ax[0].axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='-.', color=colors[index])
    #ax[0].axis([0, 80, 0, 0.02])
    ax[0].set_title('Mean Squared Error (Loss)')
    ax[0].legend()
    ax[0].grid(alpha=.3)

    ax[1].plot(history['mae'], label=f'{key} Train. acc.', alpha=.9, color=colors[index])
    ax[1].plot(history['val_mae'], label=f'{key} Valid. acc.', alpha=.9, color=colors[index], ls='--')
    ax[1].axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='-.', color=colors[index])
    #ax[1].axis([0, 80, 0, 0.10])
    ax[1].set_title('Mean Absolute Error')
    ax[1].legend()
    ax[1].grid(alpha=.3)

    ax[2].plot(history['lr'], label='Second model Learning Rate', alpha=.8, color=colors[index])
    ax[2].axvline(x=best_epoch, label='Best epoch', alpha=.3, ls='-.', color=colors[index])
    ax[2].legend()
    ax[2].grid(alpha=.3)